In [ ]:
# Running these two lines help you update networkx and pip to the latest version
!pip install networkx --upgrade
!pip install --upgrade pip

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.imagesource {
    font-size: xx-small;
}
</style>
""")

from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

import os
import networkx as nx

from IPython.core.display import HTML
def css_styling():
    styles = open("custom_style.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
# Load in the ok book
from client.api.notebook import Notebook
lab3 = Notebook('lab3.ok')
_ = lab3.auth(inline=True, force=True)

## Lab 3 - Enriching network data and testing a hypothesis about homophily

## Attributes of nodes

We have seen that there is a lot we can learn about social networks by just studying their structure---i.e., by investigating the patterns of nodes and edges. However, since social networks are typically made of people, we know that only studying their structure leaves a lot of potentially important information out of our analysis. People are not all the same, and so reducing people to identical nodes in a network can be too simplistic to accurately learn about the social processes that a network is a part of.

Today, we're going to learn how to enrich our networks by adding additional information about who is in the nodes. We'll continue to work with the theoretical idea of homophily, which we have discussed in a few previous classes.

# Measuring homophily

In class, we talked about one approach to measuring homophily in a network that is composed of nodes that are either girls or boys.

Suppose that a proportion $p$ of the nodes in the network are boys, and that a proportion $q$ of nodes in the network are girls. Then we reasoned that if we pick an edge at random from the network, and randomly assign genders to the nodes at either end of the edge, we'd see one of the following three situations:

<img src='random_gender_withfrac.jpg' width='30%'></img>

We'd expect to see an edge joining two boys with probability $p^2$; an edge joining two girls with probability $q^2$; and an edge that joins a boy and a girl with probability $pq + qp = 2p$. The approach we talked about in lecture was based on comparing the observed fraction of boy-girl edges to the value $2pq$ that would be predicted under this first null model.

Although this analysis is a good starting point, it makes an important assumption: it assumes that, on average, boys and girls have the same average network size. (If you are curious about why the model implies this, please see the optional footnote at the end of the lab.)

In this lab, we'll explore a more sophisticated metric -- the assortativity coefficient -- that accounts for differences in nodes' network sizes. The assortativity coefficient can also be extended to measure homophily on characteristics that have more than two categories.

### The assortativity coefficient

If a network's nodes can be divided into discrete groups, then a quantitative metric for the amount of homophily in a network was proposed by [Newman](https://arxiv.org/pdf/cond-mat/0205405.pdf). We'll work with this metric today.

* suppose that the nodes can all be divided into discrete groups
* let $a_i$ be the fraction of ends of edges ('stubs') in a network that are connected to nodes in group $i$
* let $e_{ij}$ be the fraction of edges in a network ('stubs') that connect nodes in group $i$ to nodes in group $j$


Newman's *assortativity coefficient* is:

$$r = \frac{\sum_i e_{ii} - \sum_i a_i^2}{1 - \sum_i a_i^2}$$

* when $r=0$, then there is no assortativity
* when $r>0$, then there is assortativity -- i.e., members of the same group tend to be more connected to each other than to other groups. At the extreme, when $r=1$, all network connections are within groups
* when $r<1$, then there is disassortativity -- i.e., members of the same group are less likely to be connected to each other than to other groups

For two groups, we can write the assortativity coefficient as

$$r = \frac{e_{11} + e_{22} - a_1^2 - a_2^2}{1 - a_1^2 - a_2^2}$$

### Example 1

<img src='example_network_groups3.png'>

**Practice** Calculate the assortativity coefficient in the example network:

In [ ]:
#         degree  group
# node 1    1     blue
# node 2    2     red
# node 3    5     blue
# node 4    1     red
# node 5    2     blue
# node 6    1     red
# node 7    2     red
# node 8    2     blue

In [ ]:
# STUB COUNTS

#       blue  red
# blue   4     6
# red    6     0

In [ ]:
# STUB TOTALS

#       total degree  a_i
# blue       10       10/16
# red        6        6/16

In [ ]:
# e_ij MATRIX

#       blue  red
# blue  4/16  6/16
# red   6/16  0

**ANSWER:** *assortativity coefficient*
$r$ = [((4/16) + 0) - (10/16)^2 - (6/16)^2] / [1 - (10/16)^2 - (6/16)^2] = -0.6

In [ ]:
(((4/16) + 0) - (10/16)**2 - (6/16)**2) / (1 - (10/16)**2 - (6/16)**2)

 Now we'll use the `networkx` package to check the results of the calculation we performed above. This will  give us a chance to learn about how the package handles node attributes.

**Question** Create a `networkx` graph called `test` that represents the example network above.<BR>
*[Hint: you can enter this network as an edgelist.]*
*[Hint: you can go back to lab 2 and see how to create a networkx graph.]*

In [ ]:
test = ...

In [ ]:
_ = lab3.grade('q1')

In [ ]:
# Let's take a look at the network you generate, is it the same as the one above?
nx.draw_circular(test, with_labels=True)

The pattern below shows us how to tell `networkx` that there is a node attribute called 'color', and it shows us how to explicitly add the color of each node:

In [ ]:
nx.set_node_attributes(test,                    
                       {1 : 'blue',
                        2 : 'red',
                        3 : 'blue',
                        4 : 'red',
                        5 : 'blue',
                        6 : 'red',
                        7 : 'red',
                        8 : 'blue'},
                      'color')

node_color=[x[0] for x in nx.get_node_attributes(test, 'color').values()]

nx.draw_circular(test, with_labels=True,node_color=node_color)

See the help page for [`set_node_attributes`](https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.classes.function.set_node_attributes.html) for more information on working with attributes.

You can check the matrix of $e_{ij}$ values by using the function `nx.attribute_mixing_matrix`:

In [ ]:
nx.attribute_mixing_matrix(test, 'color')

Note that, by default, the order of the columns of the mixing matrix is not specified. If you want to be able to interpret the columns, you can pass in a dictionary that maps the different attributes to column numbers like we do below:

In [ ]:
nx.attribute_mixing_matrix(test, 'color', mapping = {'red' : 1, 'blue' : 0})

We can calculate the value of the assortativity coefficient using the `nx.attribute_assortativity_coefficient` function:

In [ ]:
nx.attribute_assortativity_coefficient(test, 'color')

**Practice** Look at the next example network. Does it look more or less homophilous by color than the one above?

<img src='example_network_groups2.png'>

**Hand-writing Question** Calculate the assortativity coefficient for this network by filling in the missing quantities below:

In [ ]:
#         degree  group
# node 1    ...
# node 2   
# node 3   
# node 4  
# node 5    
# node 6    
# node 7    
# node 8   

In [ ]:
# STUB COUNTS

#       blue  red
# blue   ...
# red    

In [ ]:
# STUB TOTALS

#       total degree  a_i
# blue    ...
# red    

In [ ]:
# e_ij MATRIX

#       blue  red
# blue  ...
# red   

**Hand-writing Question** Calculate the *assortativity coefficient* of this network.

In [ ]:
q2 = ...
q2

In [ ]:
_ = lab3.grade('q2')

**Question** Now check your answer using the `networkx` package. Build a networkx graph called `test2` first, and then assign the color attributes to the nodes, and finally use the functions to calculate the assortativity coefficient.

In [ ]:
# Create test2 network graph
test2 = ...

In [ ]:
# Assign the attribute to the network
nx.set_node_attributes(test2,
                       ...)

In [ ]:
# Get the assortativity coefficient
q3 = ...
q3

In [ ]:
_ = lab3.grade('q3')

### Reading Add Health attributes

Recall that the Add Health dataset we have been working with stores information about network connections in an edge list.  The Add Health dataset also has information about the nodes; it has their grade, their gender, and their race/ethnicity. The code below will read all of this information in for you.

There are some problems with two of the Add Health networks' node data, so we'll leave those two out. Our set of Add Health networks is thus 82 nodes instead of 84 for today.

In [ ]:
data_path = "../../data/add-health"

def read_add_health_edges(network_id, path=data_path):
    """
    network_id : integer from 1 to 84
    
    read in the Add Health network corresponding to the given id number and
    return it as an undirected networkx object
    """

    # this file was downloaded from
    # http://moreno.ss.uci.edu/data.html#adhealth
    edge_file = os.path.join(path, "comm" + str(network_id) + ".dat")
    with open(edge_file, 'r') as f:
        edge_lines = f.readlines()
        
    network = nx.parse_edgelist(edge_lines, nodetype=int, data=[('activity_level', float)])
    
    # note that we call the to_undirected method to ensure we get an undirected network
    return(network.to_undirected())

def read_add_health_attributes_oneperrow(network_id, net, path=data_path):
    """
    Read in an Add Health attributes file that has one attribute per row
    """
    att_file = os.path.join(path, "comm" + str(network_id) + "_att.dat")
    with open(att_file, 'r') as f:
        att_lines = f.readlines()
    
    # the first 8 lines are meta-info and not actual data
    att_lines = att_lines[8:]
    
    node_races = {}
    node_grades = {}
    node_sexes = {}

    for cur_id in net.nodes():
        
        print("starting node ", cur_id)
        
        # the attributes are stored one per line for each node in sequence (race / sex / grade)
        # so line 0 is node 1's race, line 2 is node 1's sex, line 3 is node 1's grade, line 4 is node 2's race, etc
        start_idx = (cur_id-1) * 3
        this_race = str.split(g_att[start_idx])[2]
        this_sex = str.split(g_att[start_idx+1])[2]
        this_grade = str.split(g_att[start_idx+2])[2]
    
        node_races[cur_id] = this_race
        node_grades[cur_id] = this_grade
        node_sexes[cur_id] = this_sex
    
    nx.set_node_attributes(net, 'race', node_races)
    nx.set_node_attributes(net, 'grade', node_grades)
    nx.set_node_attributes(net, 'sex', node_sexes)
    
    return(net)

def read_add_health_attributes(network_id, net, path=data_path):
    """
    Read in an Add Health attributes file that has one row per node
    """
    
    # open up the attributes datafile
    att_file = os.path.join(path, "comm" + str(network_id) + "_att.dat")
    with open(att_file, 'r') as f:
        att_lines = f.readlines()
        
    # the first several lines are meta-info and not actual data;
    # the data start once we see "DATA:\n"
    header_start = att_lines.index("COLUMN LABELS:\n") + 1
    header_end = att_lines.index("DATA:\n")
    data_start = header_end + 1
    
    # build up a list that maps column index to column name
    col_defs = []
    # build up a dict that has the data for each variable
    col_data = {}
    
    for colindex, lineidx in enumerate(range(header_start, header_end)):
        # strip off the newline and the starting/ending quotes of the column name
        this_name = (str.strip(att_lines[lineidx])[1:-1]).lower()
        col_defs.append(this_name)
        # initialize the data for this column to empty dict
        col_data[this_name] = {}  
    
    att_lines = att_lines[data_start:]
    
    # for each row (corresponding to one node's data)
    # split the columns up and stick them into the appropriate
    # dict, with node id as key and attribute value as value
    for cur_id in net.nodes():
        #print('starting node ', cur_id)
        these_data = str.split(att_lines[cur_id - 1])
        
        for colidx, dat in enumerate(these_data):
            col_data[col_defs[colidx]][cur_id] = dat

    # take the data and assign it to the nodes in the graph object
    for var in col_defs:
        nx.set_node_attributes(net,  col_data[var],var)
    
    return(net)

def read_add_health_network(network_id):
    
    this_net = read_add_health_edges(network_id)
    #this_net = read_add_health_attributes(network_id, this_net)
    this_net = read_add_health_attributes(network_id, this_net)
    
    return(this_net)

Having loaded those functions, we can use the `read_add_health_network` function to load the datasets into memory:

In [ ]:
# EXCEPTIONS: networks 1 and 48 have formatting problems, so we'll omit them today
# this will take a few secs

add_health_ids = [x for x in range(2, 85) if x != 48]
add_health_networks = [read_add_health_network(x) for x in add_health_ids]

### Exploring network attributes

We'll start by looking at the fraction of students in one school that is male.

Looking at the [dataset information](http://moreno.ss.uci.edu/data.html#adhealth), you can see that the Add Health sex variable has the values 1=male, 2=female, and 0=unreported.

**Practice** Use the `get_node_attributes` function to grab the sexes of the students in the first school by filling in the code below.

In [ ]:
net = add_health_networks[0]
sexes = nx.get_node_attributes(net, 'sex')
sexes

It turns out that the `get_node_attributes` function returns a dictionary, but we will find it easier to work with `numpy` arrays. To convert the dictionary values into an array, use the following code:

In [ ]:
sexes_array = np.array(list(sexes.values()))

**Question** Calculate the proportion of students in the first school that is male.
*Hint: you can go back to lab 1 and check how to calculate proportions.

In [ ]:
q4 = ...

In [ ]:
_ = lab3.grade('q4')

**Practice** Now write a loop that calculates the fraction of students that is male in each of the Add Health schools. Store your results in an array called `add_health_frac_male`.

In [ ]:
# Like we did in lab2, create an empty array first.
add_health_frac_male = make_array()

# Then we calculate the fractions and use the for loop to accumulate the records for each network.
for net in add_health_networks:
    sexes = nx.get_node_attributes(net, 'sex').values()
    sexes = np.array(list(sexes))
    net_male_frac = np.mean(sexes == '1')
    add_health_frac_male = np.append(add_health_frac_male, net_male_frac)
    
add_health_frac_male

**Practice** Make a histogram that shows the distribution of the fraction male across the schools in the sample.

In [ ]:
Table().with_column('frac_male', add_health_frac_male).hist()

**Question** Following the pattern you used to calculate the fraction male in each community, write another loop that calculates the assortativity coefficient for sex in each community. Store your results in an array called `add_health_r`.

In [ ]:
add_health_r = ...

for net in add_health_networks:
    net_r = ...
    ...
    
add_health_r

**Practice** Plot a histogram of the assortativity coefficients across the Add Health networks.

In [ ]:
Table().with_columns('r', add_health_r).hist()

### Generating random networks with random sexes

This histogram of assortativity coefficients you just made shows the values that assortativity takes on across all of the different school networks. It seems to suggest that, across all of these networks, the assortativity coefficient for sex tends to be positive.

Now let's take a deeper dive, focusin on a specific Add Health network:

In [ ]:
first_add_health = add_health_networks[0]

We'll now go through an analysis to try to assess more rigorously whether or not there is evidence for homophily by sex in this network.

**Question** Calculate the assortativity coefficient for sex in `first_add_health`.

In [ ]:
q5 = ...
q5

In [ ]:
_ = lab3.grade('q5')

We're interested in understanding whether or not there is homophily according to sex in this specific network. The assortativity coefficient is positive, which suggests that there is evidence in favor of homophily. However, we're in a similar situation to the example we saw in lecture: it seems possible that, actually, there is no homophily by sex in this network; rather, the network is assembled as the result of a random process and, just by chance, we happened to get a network that had a positive assortativity coefficient.

To assess how likely this possibility is, we will set up a *null model* and to compare what we see in the real world to what we would see if the null model were true. Remember that a null model describes the world in the absence of the phenomenon we are interested in; here, it describes a world in which networks are formed without any homophily by sex. The first_add_health network would be the observed network.

For this null model, we will assume that the network structure is fixed, but that the gender of each node is randomly assigned. (Another way of saying this is that we *condition on* the observed network structure.)

We'd like to know what the distribution of assortativity coefficients would look like under this null model. Then we can see how likely (or unlikely) the observed assortativity coefficient would be if the null model were true.

In [ ]:
import random

def shuffle_attribute(net, att): # the two inputs are: the network, and the attribute we work with
    att_dict = nx.get_node_attributes(net, att) # get the dictionary of the network
    
    # we want a new copy of the network (we don't want to clobber the original one)
    newnet = net.copy()
    
    # create a dictionary mapping node id to shuffled attribute values
    node_ids = att_dict.keys()
    att_vals = list(att_dict.values())
    random.shuffle(att_vals)
    
    new_att = dict(zip(node_ids, att_vals)) # create the new dictionary
    
    # assign the newly shuffled attribute values
    nx.set_node_attributes(newnet, name=att, values=new_att)
    
    return(newnet)

In [ ]:
### If you have trouble understanding the function above, you could run the following codes and see what they do
# dictionary = nx.get_node_attributes(test2, 'color')
# dictionary

In [ ]:
# dictionary.keys()

In [ ]:
# list(dictionary.values())

If you run the following cell a few times, you should see that the sex value is getting shuffled

In [ ]:
test = shuffle_attribute(first_add_health, 'sex')
test.nodes[1]['sex']

**Question** Fill in the code below to reshuffle sex in the `first_add_health` network 1000 times. Record the assortativity coefficient for each reshuffled network.

In [ ]:
first_add_health = ... # use the correct index

# create an empty array for the null network
...

for _ in range(1000):
    cur_net = ... # shuffle the sex for the current network
    null_network_r = ... # record the coefficient of the shuffled current network by appending it to the array


**Practice** Make a histogram of the `er_network_r` values. What value is the assortativity coefficient centered on?

In [ ]:
Table().with_column('null_network_r', null_network_r).hist()

**Question** Now calculate the assortativity coefficient for the observed network (the first Add Health network) that you based the random networks on. Call this `observed_r`.

In [ ]:
observed_r = ...
observed_r

In [ ]:
_ = lab3.grade('q6')

**Question** Where in the null distribution does the observed value fall? What does this suggest about how likely the observed value is to have been generated from the null model?

<div class='response'>
[Answer here]
</div>

<div class='solution'>
The observed value is pretty far out in the right tail, suggesting that it is very unlikely under the null model.
</div>

**Question** We're going to use these random networks as the null model for a hypothesis test. In words, describe what our null model says about the world (the network).<BR>
*[Hint: you can look at the [Data 8 lecture slides](http://data8.org/fa16/lectures/lec19.pdf) if you don't remember what the null model is.]*

<div class='response'>
[Answer here]
</div>

<div class='solution'>
The network is the same as the one we observed, but sex is randomly assigned (in the same numbers we observe).
</div>

**Question** Now, in words, describe what the alternative (observed) model says about the world (the network).

<div class='response'>
[Answer here]
</div>

<div class='solution'>
The alternative model says that the actual observed assortativity coefficient for sex is more extreme (bigger) than the assortativity coefficient when sex is randomly assigned.
</div>

**Practice** Now calculate an empirical $p$ value based on your null and alternative hypotheses.

In [ ]:
emp_p_value = np.mean(null_network_r >= observed_r)
emp_p_value

**Question** What do you conclude about homophily by sex in this network?

In [ ]:
<div class='response'>
[Answer here]
</div>

<div class="solution">
The simulations show that in networks with the same structure as our observed network, but in which sex is randomly assigned, the  assortativity coefficient for sex has about a 0.3% chance of being more extreme than the observed assortativity in the Add Health network.  
So, using a conventional 5% threshold, we would reject the null hypothesis.
</div>

### Optional footnote

To see why the simple null model we discussed in lecture implies that the average degree of girls and boys is the same, let's start by recalling that the average degree among girls can be written as

$$
\text{avg. degree among girls} = \frac{\text{sum of girls' degrees}}{\text{number of girls}}
$$

and a similar expression holds for the average degree among boys.

Now let's think about how much each of the three types of edges (b-b, b-g, and g-g) will add to the total degrees:

| edge type | probability | added to boys' total degree | added to girls' total degree |
|-----------|-------------| -----------------|---------------------------|
| b-b | $p^2$ |  2 | 0 |
| b-g | $2pq$ | 1 | 1 |
| g-g | $q^2$  | 0 | 2 |

Using the table above, we can determine the expected total degree of girls under this model: if we draw $k$ degrees in total, then we expect $q^2 k$ g-g edges and $2pq k$ g-b edges. So we have

$$
\text{expected total girls' degree} = 2 q^2 k + 2pqk = 2qk(q + p) = 2qk.
$$

The last step follows because $q = 1-p$, so that $q+p = 1$.
To calculate, the average degree among girls, we need to divide this total degree by the number of girls. If there are $N$ people in the population, then we expect $Nq$ to be girls (since $q$ is the proportion of the population that is girls). So this means

$$
\text{expected avg. degree among girls} = \frac{2qk}{Nq} = \frac{2k}{N}.
$$


Using the same logic, we can determine that 

$$
\text{expected total boys' degree} = 2 p^2 k + 2pqk = 2pk(q + p) = 2pk.
$$

and so

$$
\text{expected avg. degree among boys} = \frac{2pk}{Np} = \frac{2k}{N}.
$$

This is the same as the expected average degree among girls.


### Rerun the tests and submit your lab

In [ ]:
import os
print("Running all tests...")
_ = [lab3.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]
print("Finished running all tests.")

In [ ]:
_ = lab3.submit()